In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 3.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd 'drive/My Drive/Colab Notebooks/matrix/matrix'

/content/drive/My Drive/Colab Notebooks/matrix/matrix


In [0]:
ls

data/       day4.ipynb      day5.ipynb  README.md
day3.ipynb  day4meta.ipynb  LICENSE     TestColabToGitHub.ipynb


In [0]:
df=pd.read_csv("data/men_shoes.csv",low_memory=False)
df.shape

(18280, 48)

In [0]:
def run_model(feats, model=RandomForestRegressor(max_depth=5)):
  X=df[feats].values
  y=df['prices_amountmin'].values

  model=model

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat']=df.brand.map(lambda x: str(x).lower()).factorize()[0]

In [0]:
model=RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.sample(5)

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
4538,AVpfAIFQilAPnD_xSxCg,NaN,Coach,"Clothing, Shoes & Accessories,Men's Accessorie...",NaN,NaN,2016-03-23T23:40:52Z,2016-03-24T02:17:33Z,NaN,NaN,8.880674e+11,"[{""key"":""Style"",""value"":[""Pouch""]},{""key"":""Mat...",NaN,http://i.ebayimg.com/images/g/sNQAAOSw91NTuFXa...,NaN,"0888067443462,888067443462,coachheritageleathe...",NaN,NaN,"[{""dateSeen"":[""2016-03-23T00:00:00Z""],""name"":""...",Coach Heritage Leather Zip Tablet Business Por...,106.95,106.95,NaN,NaN,New with tags,NaN,USD,2016-03-23T23:40:52Z,2016-03-23T00:00:00Z,NaN,True,shopdesignerdiscountsshopdesignerdiscounts - E...,NaN,NaN,FREE,NaN,NaN,http://www.ebay.com/itm/COACH-HERITAGE-LEATHER...,NaN,NaN,NaN,NaN,"[{""sourceURLs"":[""http://www.ebay.com/itm/COACH...",http://www.ebay.com/itm/COACH-HERITAGE-LEATHER...,8.880674e+11,NaN,NaN,NaN,158
2942,AVpe_cO4LJeJML43z9R8,NaN,Easy USA,"All Men's Shoes,Shoes,Men's Shoes,Clothing",Multicolor,NaN,2017-01-12T06:17:05Z,2017-01-12T06:17:05Z,"[{""dateSeen"":[""2017-01-12T06:17:05Z""],""sourceU...",NaN,1.255220e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/98527996-0c88...,NaN,"0125522036686,easyusa/m3668,125522036686",NaN,M3668,"[{""dateSeen"":[""2017-01-12T06:17:05Z""],""name"":""...",Easy Usa M3668 Mens Soft Insole Flip Flops - 3...,176.07,176.07,NaN,NaN,NaN,NaN,USD,2017-01-12T06:17:05Z,2016-11-09T00:00:00Z,NaN,True,NaN,REDUCED USD 52.95,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Easy-USA-M3668-Mens...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Easy-USA-M3668-Mens...,1.255220e+11,NaN,NaN,NaN,610
3808,AVpfNxoZ1cnluZ0-gWts,NaN,MUK LUKS,"All Men's Shoes,Shoes,Men's Shoes,Clothing",Camel,NaN,2017-01-20T18:58:26Z,2017-01-20T18:58:26Z,"[{""dateSeen"":[""2017-01-20T18:58:26Z""],""sourceU...",6.0 in x 6.0 in x 1.0 in,3.397714e+10,"[{""key"":""Primary Color"",""value"":[""Brown""]},{""k...",NaN,https://i5.walmartimages.com/asr/aa98c3aa-8cea...,NaN,"mukluksmensrossshoes/554233412,033977143531,00...",Reliable of Milwuakee,0003397714353,"[{""dateSeen"":[""2017-01-20T18:58:26Z""],""name"":""...",Muk Luks Men's Ross Shoes,85.00,85.00,NaN,NaN,new,NaN,USD,2017-01-20T18:58:26Z,2016-11-05T00:00:00Z,NaN,False,Walk Into Fashion - Walmart.com,CLEARANCE USD 42.64,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/45800864,NaN,NaN,NaN,"10,9,13,11,12","[{""sourceURLs"":[""https://www.walmart.com/ip/MU...","https://www.walmart.com/ip/45800864,https://ww...",3.397714e+10,NaN,NaN,NaN,86
4425,AVpfLF_pLJeJML4338bk,NaN,Silver Jeans,"All Men's Clothing,Men's Clothing,Clothing, Sh...",Olive,NaN,2016-03-11T08:33:43Z,2016-04-06T00:13:29Z,NaN,NaN,NaN,NaN,NaN,http://c.shld.net/rpx/i/s/pi/mp/11862/prod_856...,NaN,silverjeans/21m42995sch016,NaN,21-M42995SCH016,NaN,Silver Jeans Denim Mens Eddie Relaxed Distress...,90.00,90.00,NaN,NaN,NaN,NaN,USD,2016-03-11T08:33:43Z,2016-03-31T00:00:00Z,NaN,False,Sears.com,NaN,NaN,NaN,NaN,NaN,http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN,"32,36,34","[{""sourceURLs"":[""http://www.sears.com/content/...",http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN,NaN,791
14415,AVpfjA3O1cnluZ0-mtgu,NaN,Lee,"Clothing, Shoes & Accessories,Men's Clothing,J...",NaN,NaN,2016-09-27T02:13:52Z,2016-10-05T15:28:03Z,"[{""dateSeen"":[""2016-10-05T15:28:03Z""],""sourceU...",NaN,4.032627e+10,"[{""key"":""Style"",""value"":[""Relaxed""]},{""key"":""I...",NaN,http://i.ebayimg.com/images/g/qaIAAOSwqfNXn7ZL...,NaN,"0040326268029,04032626802

In [0]:
def parse_features(x):
  output_dict={}
  if str(x)=='nan': return output_dict
  features= literal_eval(x.replace('\\"','"'))
  for i in features:
    key=i['key'].lower().strip()
    value=i['value'][0].lower().strip()
    output_dict[key]= value
  return output_dict

df['features_parsed'] = df.features.map(parse_features)

In [0]:
keys =set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [0]:
def names(key):
  return 'cat_'+key

for key in tqdm_notebook(keys):
  df[names(key)]=df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'cat_manufacturer sku', 'cat_polarized?', 'cat_gps', 'cat_lens color',
       'cat_fabric care instructions', 'cat_closure', 'cat_crown',
       'cat_is lined', 'cat_is orthopedic', 'cat_sizebridge'],
      dtype='object', length=526)

In [0]:
df[df['cat_athlete'].isnull()].shape

(18272, 526)

In [0]:
keys_stat={}
for key in keys:
  keys_stat[key]=(df[False == df[names(key)].isnull()].shape[0]/df.shape[0] *100)


In [0]:
{k:v for k,v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
# df['cat_brand_cat']=df.cat_brand.map(lambda x: str(x).lower()).factorize()[0]

# df['cat_color_cat']=df.cat_color.map(lambda x: str(x).lower()).factorize()[0]

# df['cat_gender_cat']=df.cat_gender.map(lambda x: str(x).lower()).factorize()[0]

# df['cat_manufacturer part number_cat']=df['cat_manufacturer part number'].map(lambda x: str(x).lower()).factorize()[0]

# df['cat_material']=df.cat_material.map(lambda x: str(x).lower()).factorize()[0]

# df['cat_sport']=df.cat_sport.map(lambda x: str(x).lower()).factorize()[0]

# df['cat_style']=df.cat_style.map(lambda x: str(x).lower()).factorize()[0]

# df['cat_condition']=df.cat_condition.map(lambda x: str(x).lower()).factorize()[0]

for key in keys:
  df[names(key)+'_cat2']=df[names(key)].map(lambda x: str(x).lower()).factorize()[0]

In [0]:
feats_cat=[x for x in df.columns if 'cat2' in x]
feats_cat

['cat_color_cat2',
 'cat_year made_cat2',
 'cat_item width_cat2',
 'cat_leather grade_cat2',
 'cat_spikes type_cat2',
 'cat_retail price_cat2',
 'cat_full product manual_cat2',
 'cat_heat zones_cat2',
 'cat_animal type_cat2',
 'cat_waist size_cat2',
 'cat_target audience_cat2',
 'cat_finish_cat2',
 'cat_mpn_cat2',
 'cat_sports team_cat2',
 'cat_purpose_cat2',
 'cat_style code_cat2',
 'cat_international shipping?_cat2',
 'cat_sleeve style_cat2',
 'cat_is dark sky-compliant_cat2',
 'cat_waist_cat2',
 'cat_chronograph_cat2',
 'cat_domestic shipping_cat2',
 'cat_band type_cat2',
 'cat_assembled in country of origin_cat2',
 'cat_shape_cat2',
 'cat_country of manufacture_cat2',
 'cat_power type_cat2',
 'cat_characteristics_cat2',
 'cat_safety features_cat2',
 'cat_bag size range_cat2',
 'cat_euro size_cat2',
 'cat_fits models_cat2',
 'cat_band width_cat2',
 'cat_charge time (minutes)_cat2',
 'cat_chain length_cat2',
 'cat_measurements:_cat2',
 'cat_recommended use_cat2',
 'cat_transactionid_

In [0]:
model=RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
cols=['cat_brand_cat','cat_gender_cat', 'cat_material','cat_style','cat_condition']
cols+=feats_cat
cols=list(set(cols))
run_model(cols, model)

(-60.51220926536894, 4.322181124262401)

In [68]:
X= df[cols].values
y=df.prices_amountmin.values

m=RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)

m.fit(X,y)

perm=PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=cols)



Weight,Feature
0.0275 ± 0.0029,cat_material
0.0240 ± 0.0030,cat_material_cat2
0.0146 ± 0.0020,cat_adjustable_cat2
0.0137 ± 0.0010,cat_weight_cat2
0.0128 ± 0.0026,cat_color_cat2
0.0103 ± 0.0021,cat_casual & dress shoe style_cat2
0.0095 ± 0.0015,cat_resizable_cat2
0.0079 ± 0.0015,cat_brand_cat2
0.0050 ± 0.0006,cat_brand_cat
0.0050 ± 0.0012,cat_style
